# Time series data

In this session we will demonstrate some of the many options of Pandas to import and manipulate time series data. The objective is to read data from two different sources and combine them to obtain a time series with daily values of water level, air temperature, relative humidity and rainfall.

As usual we start by importing the required libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

We will also select a time interval for the time series by defining the start and end dates. The `to_datetime` function converts the string provided to a `datetime64` data type, which is the underlying data type in Pandas for time series data. For more information see <A href="https://pandas.pydata.org/docs/user_guide/timeseries.html">https://pandas.pydata.org/docs/user_guide/timeseries.html</A>. Note that the `to_datetime` function can also accept other arguments than a string type, see <A href="https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html">https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html</A>.

In [ ]:
t_min = pd.to_datetime("2013-08-25")
t_max = pd.to_datetime("2013-12-15")

Let's try to read the data in water_level_example.csv. The `read_csv` method has multiple options for figuring out the date and time based on the values provided in the file. One method that the documentation (<A href="https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html">https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html</A>) suggests is the following

In [ ]:
df0 = pd.read_csv("data/water_level_example.csv", parse_dates=["Date", "Time"])
print(df0.head())

However, that somehow does not seem to have worked: The 'Date' and 'Time' column appear exactly as they are in the file and the index is a series of integer numbers. What we wanted instead was the 'Date' and 'Time' columns to be merged into a single column that has the datetime64 data type. Since this has not worked we need to try something different (usually you'll try to look for an answer on <A href="https://stackoverflow.com/">https://stackoverflow.com/</A> in a situation like this one). For example

In [ ]:
df0 = pd.read_csv("data/water_level_example.csv")
df0 = df0.set_index(pd.to_datetime(df0["Date"].astype(str) + " " + df0["Time"].astype(str)))
print(df0.head())

That looks more like it: The index now contains the datetime info. We can delete the original 'Date' and 'Time' columns to avoid storing redundant information. In fact, the 'ms' column doesn't look particularly useful, so let's also delete that one. The `drop` looks in the DataFrame's index by default, so it will delete rows for which a match is found with any of the names provided. Because we want to delete columns, and not rows, we have to override the default behaviour by passing `axis=1`.

In [ ]:
df0 = df0.drop(["Date", "Time", "ms"], axis=1)
print(df0.head())

Let's plot the 'LEVEL' column to visualise the water level measurements.

In [ ]:
df0['LEVEL'].plot()

We can also zoom in to a shorter time period. Note that the dates can be passed as strings. Matplotlib understands these, which makes working with time series graphs a lot easier than it was years ago.

In [ ]:
ax = df0['LEVEL'].plot()
ax.set_xlim("2013-09-15", "2013-10-15")

We notice that there is a gap in the data on the first of October. There are many ways to deal with missing data, but for this particular example it seems justifiable to use linear interpolation to fill the gap. Pandas provides the `interpolate` method for this, and all we have to do is call it without any arguments

In [ ]:
df0 = df0.interpolate()
ax = df0['LEVEL'].plot()
ax.set_xlim("2013-09-15", "2013-10-15")

Indeed the gap is gone. The values stored in the csv file are water heights above the pressure transducer, so they still need to be converted to water levels with respect to the reference marker on the jetty pole. We do this by shifting the data series by fitting it through the manual water level measurements in the fille `manual_readings.xlsx`, which are expressed with respect to the local reference point. Note that the manual measurements in the file are in cm above the reference point, so we divide by -100 to make sure they are in m and decrease in time, just like the water colum height measured by the transducer is decreasing with time.

In [ ]:
dfm = pd.read_excel("data/manual_readings.xlsx", index_col=0, parse_dates=True)
# df0['LEVEL'] = -df0['LEVEL']
df0['manual'] = dfm / -100.

fig, ax = plt.subplots()
ax.plot(df0.index, df0['LEVEL'])
ax.plot(df0.index, df0['manual'], 'o')

One way to calcuate the required offset is to calculate the mean of the differences between the manual and the pressure transducer readings. Note that the column `manual` in `df0` contains many NaN values, so the `nanmean` method from NumPy is used so that these are ignored in calculating the mean.

In [ ]:
wl_offset = np.nanmean(df0['manual'] - df0['LEVEL'])

fig, ax = plt.subplots()
ax.plot(df0.index, wl_offset + df0['LEVEL'])
ax.plot(df0.index, df0['manual'], 'o')

Now that the water levels are stored in the right way, we can move on to read the weather data. The problem to tackle with these is that they are spread across multiple worksheets in an Excel file, with each worksheet containing the data of a particular month. Fortunately it is not too difficult to merge the data from the various sheets into a single DataFrame. The first thing to do is to call the Pandas method `read_excel` and explicitly pass `sheet_name=None` as an argument. Note that the other two arguments ensure that the dates are parsed correctly.

In [ ]:
sheets_dict = pd.read_excel(
    "data/weather_data_by_month.xlsx", 
    index_col=0, 
    parse_dates=True,
    sheet_name=None,
)

The effect of passing `sheet_name=None` is that `read_excel` returns a dictionary with the worksheets. Each worksheet is read as a DataFrame and is accessible in the dictionary by passing the sheet name as a key. So by iterating over the dictionary items, we can grab each DataFrame. By defining an empty DataFrame first, each DataFrame from `sheets_dict` can be appended using the `concat` function to create the DataFrame with all the data.

In [ ]:
dfd = pd.DataFrame()
for sheet_name, df in sheets_dict.items():
    dfd = pd.concat((dfd, df))

Now let's trim the DataFrame with weather data to the period of interest

In [ ]:
idx = (dfd.index >= t_min) & (dfd.index <= t_max)
dfd = dfd.loc[idx]

We can combine the weather data with the water level data if we keep in mind that the water levels were reported at hourly intervals, so they must be resampled to daily values. For this, Pandas provides the `resample` function, which is called here with the argument `1D`, which indicates that the measurements need to be aggregated at a one-day interval. The `.mean()` at the end indicates that the values need to be averaged. These daily averages can be added as a new column to `dfd`

In [ ]:
dfwl = wl_offset + df0["LEVEL"].resample('1D').mean()
dfd['wl'] = dfwl

By loading the coefficients of the polynomials that calculate dam area and volume based on the water level (from session 3), we can convert the measured water levels to water surface area and volume, and add these as new columns to the DataFrame.

In [ ]:
p_func_V = np.poly1d(np.loadtxt("p_coef_V_linear.dat"))
p_func_A = np.poly1d(np.loadtxt("p_coef_A_linear.dat"))

dfd["volume"] = p_func_V(dfd["wl"])
dfd["area"] = p_func_A(dfd["wl"])


Finally, the data can be saved to an Excel file using `to_excel`.

In [ ]:
dfd.to_excel("daily_wl&meteo_data.xlsx")

## Homework: Vapour pressure deficit

The evaporative demand of the atmosphere depends on the vapour pressure deficit ($vpd$), which is the difference between the saturation vapour pressure $e_s$ and the actual vapour pressure ($e_a$):

$vpd = e_s - e_a$

where 

$ e_a = \frac{rh}{100} e_s$

in which $rh$ is the relative humidity in percent, and 

$ e_s = 0.6108 e^{\frac{17.27 T}{T + 237.3}}$

In the equations above $vpd$, $e_a$ and $e_s$ are expressed kPa and the temperature $T$ is in °C. For more details see <A href="https://www.fao.org/3/x0490e/x0490e07.htm">https://www.fao.org/3/x0490e/x0490e07.htm</A>.

***Exercise***: Use the equations above to calculate the $vpd$ with the available temperature and relative humidity data. Add the result as a column to the DataFrame `dfd` and save the DataFrame to an Excel file.